In [20]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
import sys
import math

FileNotFoundError: [Errno 2] No such file or directory: '/Users/liyutong/anaconda3/share/proj\\epsg'

In [13]:
GEO=pd.read_csv('US_Counties_Location.csv')
GEO.head()
#读取GEO，经纬度数据

,State,FIPS,County,County Seat(s),Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,AL,1001,AUTAUGA,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,32.536382,-86.644490
1,AL,1003,BALDWIN,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",30.659218,-87.746067
2,AL,1005,BARBOUR,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,31.870670,-85.405456
3,AL,1007,BIBB,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
4,AL,1009,BLOUNT,Oneonta,"57,322","1,669.96",644.776,15.157,5.852,"1,685.12",650.628,33.977448,-86.567246


In [14]:
NFLIS=pd.read_excel('MCM_NFLIS_Data.xlsx','Data')
NFLIS=NFLIS[NFLIS['YYYY']==2016]
NFLIS.head()
#读取NFLIS数据

,YYYY,State,COUNTY,FIPS_State,FIPS_County,FIPS_Combined,SubstanceName,DrugReports,TotalDrugReportsCounty,TotalDrugReportsState
16493,2016,VA,ACCOMACK,51,1,51001,Buprenorphine,2,114,33539
16494,2016,OH,ADAMS,39,1,39001,Buprenorphine,15,365,115276
16495,2016,PA,ADAMS,42,1,42001,Fentanyl,12,475,72376
16496,2016,PA,ADAMS,42,1,42001,Heroin,42,475,72376
16497,2016,PA,ADAMS,42,1,42001,Propoxyphene,1,475,72376


In [15]:
countyList=[]#一个包括所有county的list
fipsList=[]#fips，记录所有county的fips（邮编）
for i in range(NFLIS.shape[0]):
    if NFLIS.iloc[i]['FIPS_Combined'] not in fipsList:
        fipsList.append(NFLIS.iloc[i]['FIPS_Combined'])
        countyList.append([NFLIS.iloc[i]['FIPS_Combined'],
                         NFLIS.iloc[i]['COUNTY'],
                         NFLIS.iloc[i]['TotalDrugReportsCounty'],
                         float(GEO[GEO['FIPS']==NFLIS.iloc[i]['FIPS_Combined']]['Latitude']),
                         float(GEO[GEO['FIPS']==NFLIS.iloc[i]['FIPS_Combined']]['Longitude'])
                        ])#如果不在fips的list里，说明是新城市，添加drug数据和经纬度进countyList里


In [16]:
countyList=pd.DataFrame(countyList,columns=['FIPS','County','DrugReports','Latitude','Longitude'])##转化为Pandas
countyList=countyList.sort_values(by=['FIPS'])##根据FIPS排序
countyList.index = range(len(countyList))  ##重新设置index
countyList.head()

,FIPS,County,DrugReports,Latitude,Longitude
0,21001,ADAIR,115,37.105541,-85.281401
1,21003,ALLEN,107,36.750770,-86.192458
2,21005,ANDERSON,72,38.005396,-84.986417
3,21007,BALLARD,16,37.051120,-89.009178
4,21009,BARREN,289,36.963614,-85.924990


In [17]:
def rad(d):
    return d * np.pi / 180.0

def getDistance(lat1, lng1, lat2, lng2):
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(sin(a/2), 2) + cos(radLat1) * cos(radLat2) * math.pow(sin(b/2), 2)))
    s = s * 6371*1000
    return s
distanceMatrix=np.zeros([len(countyList),len(countyList)])#生成county间距离矩阵

for i in range(len(countyList)):
    #print(i)
    for j in range(len(countyList)):
        distanceMatrix[i][j]=getDistance(countyList.iloc[i,3],
                                       countyList.iloc[i,4],
                                       countyList.iloc[j,3],
                                       countyList.iloc[j,4])#计算所有county之间的直线距离

In [18]:
np.save('distanceMatrix.npy',distanceMatrix)
countyList.to_csv('countyList.csv')
distanceDF=pd.DataFrame(distanceMatrix)
distanceDF.index=countyList['FIPS']
distanceDF.columns=countyList['FIPS'].loc[:]
distanceDF.to_csv('distanceDataFrame.csv')

In [19]:
mapp = Basemap(projection='stere',lat_0=40,lon_0=-80,
            llcrnrlat=35 ,urcrnrlat=43,
            llcrnrlon=-90,urcrnrlon=-70,
            rsphere=6371200.,resolution='h',area_thresh=10000)
mapp.drawcoastlines()
mapp.drawstates()        
mapp.drawcountries()
mapp.drawcounties()
mapp.zorder=0

lat=np.array(countyList['Latitude'])
lon=np.array(countyList['Longitude'])
dR=np.array(countyList['DrugReports'])
size=(dR/np.max(dR))*np.max(dR)
x,y = mapp(lon,lat)
mapp.scatter(x,y,s=size,c=size,cmap='GnBu',alpha=0.5)

plt.rcParams['savefig.dpi'] = 100
plt.rcParams['figure.dpi'] = 100
plt.title('DrugMap')
plt.savefig('US_drug_map.eps')
plt.savefig('US_drug_map.svg')
plt.savefig('US_drug_map.png')
plt.colorbar(shrink=0.75)
plt.rcParams['figure.figsize']=[16,9]
plt.show()

NameError: name 'Basemap' is not defined

In [37]:
help(plt.colorbar)

Help on function colorbar in module matplotlib.pyplot:

colorbar(mappable=None, cax=None, ax=None, **kw)
    Add a colorbar to a plot.
    
    Function signatures for the :mod:`~matplotlib.pyplot` interface; all
    but the first are also method signatures for the
    :meth:`~matplotlib.figure.Figure.colorbar` method::
    
      colorbar(**kwargs)
      colorbar(mappable, **kwargs)
      colorbar(mappable, cax=cax, **kwargs)
      colorbar(mappable, ax=ax, **kwargs)
    
    Parameters
    ----------
    mappable :
        The :class:`~matplotlib.image.Image`,
        :class:`~matplotlib.contour.ContourSet`, etc. to
        which the colorbar applies; this argument is mandatory for the Figure
        :meth:`~matplotlib.figure.Figure.colorbar` method but optional for the
        pyplot :func:`~matplotlib.pyplot.colorbar` function, which sets the
        default to the current image.
    
    cax : :class:`~matplotlib.axes.Axes` object, optional
        Axes into which the colorbar wil

## 一些思考
- 大小和颜色代表两个维度的数据
- 大小代表影响力
- 颜色代表药物消费的报告数

## 关于药物消费
- 用药物滥用报告的数目代表消费状况是否合理
- “影响力”是否和药物消费有关